# Data Wrangling and Visualization workbook
This workbook will be used to generate the data file that will be ready for use in the Event Based Model. Along the way, you will need to implement some of the data cleaning, data wrangling and data visualization skills that you learned in the demonstration notebook using the WHO suicide data.  In the project notebook below, you will find instructions for what to do in each step followed by an empty code cell to enter your work. Don't forget that you will need to import the appropriate packages for this work.



In [1]:
# Put your import steps here
import os
import pandas as pd
import numpy as np

## Step 1 - Load in ADNIMERGE
The first step we must do is to identify the measurements that will be used as features in the Event Based Model. From [Alex Young's original paper](https://academic.oup.com/brain/article/137/9/2564/2848155), the features that are to be included are: 
* Cerebrospinal Fluid (CSF) INNO-BIA AlzBio3 immunoassay ('INNO')
  * Amyloid Beta 1-42
  * phosphorylated tau
  * total tau
* Volumetric measurements from **1.5T** magnetic resonance imaging (MRI)
  * Whole brain volume
  * Ventricular volume
  * Entorhinal cortex volume
  * Hippocampal volume
  * Middle temporal cortex volume
  * Fusiform cortex volume 
  * Annualised whole brain atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
  * Annualised hippocampal atrophy between 0 and **12 months** using Boundary Shift Integral (BSI)
* Cognitive measures
  * Mini mental state examination (MMSE)
  * ADAS-COG13
  * Rey Auditory Verbal Learning Test (RAVLT)

Many of these features should be available in the ADNI MERGE dataset. We have included both the data dictionary and the methods so that you can better understand how this spreadsheet was created and what it represents. Please load in the ADNIMERGE spreadsheet. 
Here are some of the questions:
* Which column identifies the subject?
* Can you identify the features above in the column? 
* Are any of the features above missing?


In [2]:
# Your answer to Step 1
# Below put your code that will load up the ADNI MERGE spreadsheet
data_root="/Users/davecash/Data/IDEAS/TeamCoder_EBM"
df_adni = pd.read_csv(os.path.join(data_root,"adnimerge_ideas_merge_26may2022.csv"))
df_adni = df_adni.sort_values(['RID','EXAMDATE'])
print(df_adni)
df_adni.info(verbose=True)


        RID COLPROT ORIGPROT        PTID  SITE VISCODE    EXAMDATE DX_bl  \
4         2  ADNIGO    ADNI1  011_S_0002    11     m66  04/03/2011    CN   
1         2   ADNI1    ADNI1  011_S_0002    11     m06  06/03/2006    CN   
0         2   ADNI1    ADNI1  011_S_0002    11      bl  08/09/2005    CN   
9         2   ADNI2    ADNI1  011_S_0002    11     m96  09/09/2013    CN   
11        2   ADNI2    ADNI1  011_S_0002    11    m108  13/10/2014    CN   
...     ...     ...      ...         ...   ...     ...         ...   ...   
15851  7043   ADNI3    ADNI3  035_S_7043    35      bl  21/03/2022   NaN   
15852  7045   ADNI3    ADNI3  021_S_7045    21      bl  09/03/2022   SMC   
15853  7046   ADNI3    ADNI3  941_S_7046   941      bl  06/04/2022    CN   
15854  7051   ADNI3    ADNI3  941_S_7051   941      bl  30/03/2022   SMC   
15855  7054   ADNI3    ADNI3  027_S_7054    27      bl  04/04/2022  EMCI   

        AGE PTGENDER  ...  FBB_bl  Years_bl   Month_bl Month    M  \
4      74.3     Ma

/Users/davecash/anaconda3/envs/teamcoder_ebm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (20,105,116) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Step 2 - Filter data
The event based model primarily works on cross-sectional data. In the data, the features you have identified from ADNIMERGE should be coming from the **baseline** visit and from scans acquired on a **1.5T** scanner.  Figure out how to filter the rows according to these two criteria. The resulting data frame should have 818 rows remaining.

In [3]:
# Your answer to Step 2
# Below put your code that will filter the ADNI MERGE spreadsheet
df_adni = df_adni[df_adni['FLDSTRENG']=='1.5 Tesla MRI']
df_adni = df_adni[df_adni['VISCODE']=='bl']
print(df_adni)


       RID COLPROT ORIGPROT        PTID  SITE VISCODE    EXAMDATE DX_bl   AGE  \
0        2   ADNI1    ADNI1  011_S_0002    11      bl  08/09/2005    CN  74.3   
16       3   ADNI1    ADNI1  011_S_0003    11      bl  12/09/2005    AD  81.3   
21       4   ADNI1    ADNI1  022_S_0004    22      bl  08/11/2005  LMCI  67.5   
27       5   ADNI1    ADNI1  011_S_0005    11      bl  07/09/2005    CN  73.7   
34       6   ADNI1    ADNI1  100_S_0006   100      bl  29/11/2005  LMCI  80.4   
...    ...     ...      ...         ...   ...     ...         ...   ...   ...   
7196  1425   ADNI1    ADNI1  041_S_1425    41      bl  17/08/2007  LMCI  75.6   
7206  1426   ADNI1    ADNI1  137_S_1426   137      bl  15/10/2007  LMCI  83.4   
7209  1427   ADNI1    ADNI1  127_S_1427   127      bl  28/08/2007  LMCI  69.6   
7230  1430   ADNI1    ADNI1  128_S_1430   128      bl  21/09/2007    AD  83.4   
7234  1435   ADNI1    ADNI1  041_S_1435    41      bl  11/09/2007    AD  82.4   

     PTGENDER  ...  FBB_bl 

## Step 3 - Merging data
You should have identified that some features from the original list that are missing in the ADNI MERGE spreadsheet. Have a look at other spreadsheets in the data folder and identify where you might find the missing data.
In order to combine data you need to *merge* the two data sets. This involves finding key identifiers (the "on" columns) that will correspond to the same subject in both spreadsheets, so that the columns can be combined together in one data frame. Please remember that we only need BSI values from 0 to 12 months, so think about what filtering you will need to do with the new spreadsheet that you are loading in before performing the merge.

In [4]:
# Your answer to Step 2
# Below put your code that will merge the ADNI MERGE spreadsheet
# with other ADNI data availale.
df_bsi = pd.read_csv(os.path.join(data_root,"ucl_bsi_ideas_merge_26may2022.csv"))
df_bsi = df_bsi.sort_values(by = ["RID","VISCODE2","REGRATING"])
df_bsi = df_bsi[df_bsi['VISCODE2']=="m12"]
df_bsi = df_bsi[df_bsi['MRFIELD']==1.5]
print(df_bsi.info(verbose=True))

df_combined = pd.merge(df_adni,df_bsi,how="left",on="RID")
print(df_combined)
print(list(df_combined.columns),sep=",")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 682 entries, 3 to 4369
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RID           682 non-null    int64  
 1   VISCODE       682 non-null    object 
 2   VISCODE2      682 non-null    object 
 3   EXAMDATE      682 non-null    object 
 4   DT            682 non-null    object 
 5   VERSION       682 non-null    object 
 6   RUNDATE       682 non-null    object 
 7   STATUS        682 non-null    int64  
 8   LONIUID       682 non-null    object 
 9   LONIUID_BASE  576 non-null    object 
 10  BRAINVOL      682 non-null    float64
 11  VENTVOL       682 non-null    float64
 12  HIPPOVOL_R    0 non-null      float64
 13  HIPPOVOL_L    0 non-null      float64
 14  DBCBBSI       576 non-null    float64
 15  KMNDBCBBSI    576 non-null    float64
 16  ANN_BBSI      633 non-null    float64
 17  VBSI          576 non-null    float64
 18  HBSI_R        0 non-null     

## Step 4 - Remove unneeded columns
After the merge, there should be way more columns than what are needed for the event based model. So we can get rid of unwanted columns that we don't need for the replication. Remember - there will be additional columns (such as identifiers or demographic information) that you will need to keep that will not serve as features in the EBM.  

In [5]:
keep_columns = ['RID', 'ORIGPROT', 'SITE', 'VISCODE_x', 'EXAMDATE_x', 'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'APOE4', 'PIB', 'AV45', 'FBB', 'ABETA_ELECSYS', 'TAU_ELECSYS', 'PTAU_ELECSYS','ABETA_INNO', 'TAU_INNO', 'PTAU_INNO', 'ADAS13', 'MMSE', 'RAVLT_immediate', 'FLDSTRENG', 'MRFIELD','IMAGEUID', 'LONIUID', 'FSVERSION','Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'TEMPQC','ICV','ANN_BBSI','ANN_HBSI','DX', 'Month', 'M','VISCODE2', 'EXAMDATE_y', 'LONIUID_BASE','REGRATING','HPACCEPT_R','HPACCEPT_L','QC_PASS']
features = [ 'ABETA_INNO', 'TAU_INNO', 'PTAU_INNO', 'ADAS13', 'MMSE', 'RAVLT_immediate','Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp','ANN_BBSI','ANN_HBSI'] 
df_combined = df_combined[df_combined['VISCODE_x']=="bl"]
df_combined = df_combined[keep_columns]
print(df_combined.info(verbose=True))
df_combined[['RID','VISCODE_x'] + features]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 0 to 817
Data columns (total 49 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RID              818 non-null    int64  
 1   ORIGPROT         818 non-null    object 
 2   SITE             818 non-null    int64  
 3   VISCODE_x        818 non-null    object 
 4   EXAMDATE_x       818 non-null    object 
 5   DX_bl            818 non-null    object 
 6   AGE              818 non-null    float64
 7   PTGENDER         818 non-null    object 
 8   PTEDUCAT         818 non-null    int64  
 9   PTETHCAT         818 non-null    object 
 10  PTRACCAT         818 non-null    object 
 11  APOE4            818 non-null    float64
 12  PIB              20 non-null     float64
 13  AV45             0 non-null      float64
 14  FBB              0 non-null      float64
 15  ABETA_ELECSYS    396 non-null    object 
 16  TAU_ELECSYS      396 non-null    object 
 17  PTAU_ELECSYS    

,RID,VISCODE_x,ABETA_INNO,TAU_INNO,PTAU_INNO,ADAS13,MMSE,RAVLT_immediate,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ANN_BBSI,ANN_HBSI
0,2,bl,NaN,NaN,NaN,18.67,28.0,44.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,NaN,NaN
1,3,bl,131.0,68.0,21.0,31.00,20.0,22.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,19.090141,0.229140
2,4,bl,256.0,42.0,13.0,21.33,27.0,37.0,39605.0,6869.0,1154980.0,3983.0,19036.0,19615.0,6.027470,0.020969
3,5,bl,115.0,112.0,68.0,14.67,29.0,37.0,34062.0,7075.0,1116630.0,4433.0,24788.0,21614.0,14.563351,0.056883
4,6,bl,NaN,NaN,NaN,25.67,25.0,30.0,39826.0,5348.0,927510.0,2277.0,17963.0,17802.0,-2.380889,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,1425,bl,NaN,NaN,NaN,17.67,28.0,16.0,13914.0,6577.0,900211.0,3045.0,14014.0,18049.0,3.979931,NaN
814,1426,bl,NaN,NaN,NaN,17.00,26.0,44.0,140133.0,6296.0,1065330.0,2298.0,13587.0,20681.0,21.152246,NaN
815,1427,bl,NaN,NaN,NaN,29.00,27.0,28.0,19314.0,8585.0,1071630.0,3575.0,15827.0,23302.0,NaN,NaN
816,1430,bl,NaN,NaN,NaN,32.00,21.0,21.0,24223.0,4109.0,777501.0,2741.0,11872.0,13371.0,NaN,NaN


## Step 5 - Identify complete case data
The Event Based Model requires all features to be present for an observation to be included. Please remove any rows where one of the features that you plan to put in the EBM has missing data.
This should be your final data set. Answer a few questions:
* For each variable, how many subjects had missing data? 
* How many subjects remain? 
* How do the subjects break down across diagnosis?
* Within diagnosis, how do they breakdown in terms of sex, age, APOE status?
* How do these numbers compare to the paper? 


In [6]:
# Your answer to Step 5
# Below put your code to remove missing data
# and answer the descriptive statistics queries
# around the final data set
# This counts the number of missing values per feature
print(df_combined[features].isna().sum())

# This removes any observations with missing data, EBM typically requires
# complete case analysis
df_complete = df_combined.dropna(how='any',subset=features)
print(df_complete.shape)

# Save the complete cases to the CSV
df_complete.to_csv(os.path.join(data_root,"ebm_with_noqc.csv"))

#Produce two way table between diagnosis and sex
dx_x_gen = pd.crosstab(df_complete['DX'],df_complete['PTGENDER'],margins=True)
print(dx_x_gen)

# Another two way table, this time between the QC variable 
# for the BSI pipeline ("REGRATING") and the QC variable
# for Freesurfer Temporal Lobe segmentations
bsiqc_x_fsqc = pd.crosstab(df_complete['REGRATING'],df_complete['TEMPQC'],margins=True)
print(bsiqc_x_fsqc)

# Keep only those that pass QC according to BSI
bsi_qc = df_complete[df_complete['REGRATING']<4]
# How many observations are left?
print(bsi_qc.shape)
# write out this version - we can test out EBM on multiple data sets
bsi_qc.to_csv(os.path.join(data_root,'ebm_with_bsiqc.csv'),index=False)

# Keep only those that pass QC according to FreeSurfer
fs_qc = df_complete[df_complete['TEMPQC']=="Pass"]
# How many observations are left?
print(fs_qc.shape)
#Write out these data sets
fs_qc.to_csv(os.path.join(data_root,'ebm_with_fsqc.csv'),index=False)

#Keep only that passed both
bsi_fs_qc = df_complete[np.all([df_complete['REGRATING']<4, df_complete['TEMPQC']=="Pass"],axis=0)]
#Get nubmer of observations and write out
print(bsi_fs_qc.shape)
bsi_fs_qc.to_csv(os.path.join(data_root,'ebm_with_bsifsqc.csv'),index=False)

# The FS QC is the one that is closest to Alex's paper
# There are three differences, a couple may have passed QC originally
# but then on further review were failed.
# This shows the demographcis that you can compare to the table.
fs_qc.groupby("DX_bl").describe()


ABETA_INNO         464
TAU_INNO           464
PTAU_INNO          464
ADAS13               8
MMSE                 0
RAVLT_immediate      4
Ventricles          14
Hippocampus          0
WholeBrain          10
Entorhinal           0
Fusiform             0
MidTemp              0
ANN_BBSI           185
ANN_HBSI           464
dtype: int64
(344, 49)
PTGENDER  Female  Male  All
DX                         
CN            49    54  103
Dementia      35    41   76
MCI           57   108  165
All          141   203  344
TEMPQC     Fail  Pass  All
REGRATING                 
1.0          13    78   91
2.0          22   118  140
3.0          16    43   59
4.0           9    44   53
5.0           1     0    1
All          61   283  344
(290, 49)
(283, 49)
(239, 49)


RID                                                               \
       count        mean         std  min     25%    50%      75%     max   
DX_bl                                                                       
AD      63.0  627.920635  423.885000  3.0  276.00  577.0  1042.50  1373.0   
CN      92.0  477.543478  343.272916  5.0  168.75  475.5   698.25  1250.0   
LMCI   128.0  696.585938  394.514883  4.0  361.75  646.5  1030.75  1423.0   

        SITE             ... HPACCEPT_L     QC_PASS                           \
       count       mean  ...        75% max   count mean  std  min  25%  50%   
DX_bl                    ...                                                   
AD      63.0  68.492063  ...        NaN NaN    63.0  1.0  0.0  1.0  1.0  1.0   
CN      92.0  58.652174  ...        NaN NaN    92.0  1.0  0.0  1.0  1.0  1.0   
LMCI   128.0  62.875000  ...        NaN NaN   128.0  1.0  0.0  1.0  1.0  1.0   

                 
       75%  max  
DX_bl            
AD     1.0  1.0  
CN     1.0  1.0  
LMCI   1.0  1.0  

[3 rows x 248 columns]

## Step 6 - Data review
We have looked at the final data set at a group level, but let's actually visualise the data. For the various features that are to be included in the EBM, create some plots of your choice to look at how these values differ between the cognitively normal individuals and those showing evidence of cognitive impairment. You can break this latter group into mild cognitive impairment and Alzheimer's disease if you wish. 

In [7]:
# Your answer to Step 6
# Below put your code for visualising the data
